In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
np.seterr(divide = 'ignore') 

df = pd.read_csv('/Users/eugenganenco/Desktop/srealtyAnalysis/DataWithCoordPrice.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.loc[:, ~df.columns.str.contains('ID')]

In [2]:
df['Swimming pool'].dtype 

dtype('float64')

In [3]:
df = df.loc[df['price'] != 0]

df.head()

,link,location,district,HouseType,total price,Update,Building,Object status,Location of the house,Home type,The floor,Built-up area,Usable area,Floor area,Land area,Garden area,Store,Move-in date,Water,Heating,Gas,Garbage,Transport,Communication,Energy efficiency of the building,Barrier-free,Equipment,companyName,proximityIndex,Index,Note on price,Electricity,Indicator of energy efficiency of the building,Object location,Parking,Garage,Year of reconstruction,Lift,Cena,Year of approval,Telecommunication,Certificate of energy performance of the building,Swimming pool,Number of flats,Housing costs,Discounted,Original price,Pool area,Ceiling height,Condition,Soil construction,Starting price,Expert opinion,Minimum bid,Auction principal,Type of auction,Place of auction,The date of the auction,Auction decree,Expert opinion.1,Inspection date,Sale start date,Minimum purchase price,The date of the 1st inspection,The date of the 2nd inspection,Date of completion of construction,Inspection date to,locationLat,locationLong,price
0,https://www.sreality.cz/detail/prodej/dum/rodinny/as-as-majakovskeho/1143850572,"Majakovského, Aš",cheb,rodinne-domy\n,4 800 000 Kč za nemovitost,Včera,Cihlová,Dobrý,Řadový,Patrový,3,44 m2,180 m2,130 m2,180 m2,136 m2,1,Ihned,Dálkový vodovod,Lokální plynové,Plynovod,Veřejná kanalizace,"Vlak, Silnice, Autobus",Asfaltová,Třída G - Mimořádně nehospodárná č. 264/2020 Sb. podle vyhlášky,0.0,Částečně,TOP REALITY,466.733333,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.2152174,12.1941903,4800000
1,https://www.sreality.cz/detail/prodej/dum/rodinny/luby-dolni-luby-/4147755356,"Luby - Dolní Luby, okres Cheb",cheb,rodinne-domy\n,4 990 000 Kč za nemovitost,Včera,Cihlová,Velmi dobrý,Samostatný,Patrový,NaN,117 m2,158 m2,NaN,1401 m2,NaN,1,NaN,"Místní zdroj, Studna",Lokální tuhá paliva,Individuální,Jímka,"Vlak, Dálnice, Silnice, Autobus",Asfaltová,Třída C - Úsporná č. 78/2013 Sb. podle vyhlášky,NaN,NaN,WOW reality,1775.000000,1,"včetně provize, včetně právního servisu","230V, 400V","58,4 kWh/m^2 za rok",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.2398432,12.4141533,4990000
2,https://www.sreality.cz/detail/prodej/dum/rodinny/ovesne-kladruby-ovesne-kladruby-/275785548,"Ovesné Kladruby, okres Cheb",cheb,rodinne-domy\n,1 199 000 Kč za nemovitost,21.10.2022,Cihlová,Před rekonstrukcí,Řadový,Patrový,2,87 m2,225 m2,160 m2,785 m2,633 m2,NaN,NaN,Studna,Lokální tuhá paliva,NaN,Septik,"Vlak, Silnice, Autobus",Asfaltová,Třída G - Mimořádně nehospodárná,NaN,Částečně,Personal Reality s.r.o,2313.000000,2,"včetně DPH, včetně poplatků, včetně provize, včetně právního servisu",400V,NaN,Klidná část obce,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.953,12.7790915,1199000
3,https://www.sreality.cz/detail/prodej/dum/rodinny/luby-luby-petra-bezruce/2112174940,"Petra Bezruče, Luby",cheb,rodinne-domy\n,7 400 000 Kč za nemovitost,21.10.2022,Cihlová,Po rekonstrukci,Samostatný,Patrový,3 včetně 1 podzemního,NaN,341 m2,NaN,381 m2,260 m2,1,NaN,Dálkový vodovod,"Lokální plynové, Ústřední plynové",Plynovod,Veřejná kanalizace,"Vlak, Dálnice, MHD, Autobus",Asfaltová,Třída G - Mimořádně nehospodárná,NaN,Částečně,Znalecká a poradenská kancelář s.r.o.,763.538462,3,"Garantujeme roční výnos 600.000, - Kč",230V,NaN,Okraj obce,1.0,NaN,2022.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.250134,12.4078522,7400000
4,https://www.sreality.cz/detail/prodej/dum/rodinny/dolni-zandov-uboci-/3171592012,"Dolní Žandov - Úbočí, okres Cheb",cheb,rodinne-domy\n,6 900 000 Kč za nemovitost (k jednání),20.10.2022,Cihlová,Velmi dobrý,Samostatný,Patrový,2,650 m2,180 m2,180 m2,1893 m2,NaN,1,NaN,Místní zdroj,"Ústřední tuhá paliva, 

### I will first perform some modifications to the dataframe that should not cause to data leakage

In [4]:
def extractInt(string):
    return re.sub('(m2)', '', string)

featureList = ['Built-up area', 'Usable area', 'Floor area', 'Land area', 'Garden area']
featuresModified = df.loc[:,featureList].applymap(extractInt, na_action='ignore')

for feature in featureList:
    df[feature] = featuresModified[feature].values
    df[feature] = df[feature].fillna(-1)
    df[feature] = df[feature].astype(int)
    df[feature] = df[feature].replace(-1, np.nan)
    
print(df.loc[:,featureList])

print(df['Built-up area'].dtypes)

      Built-up area  Usable area  Floor area  Land area  Garden area
0              44.0          180       130.0      180.0        136.0
1             117.0          158         NaN     1401.0          NaN
2              87.0          225       160.0      785.0        633.0
3               NaN          341         NaN      381.0        260.0
4             650.0          180       180.0     1893.0          NaN
...             ...          ...         ...        ...          ...
5994          129.0          240         NaN      324.0          NaN
5995          220.0          200         NaN     1303.0       1303.0
5996            NaN          100         NaN     1501.0          NaN
5997          119.0          119         NaN      626.0          NaN
5998         1000.0         1000      1000.0    28663.0          NaN

[5456 rows x 5 columns]
float64


This exrtacts the numerical value of the "area" variables.

In [5]:
notesDict = Counter()
notesList = df['Note on price'].tolist()
for note in notesList:
    if pd.isna(note):
        continue
    notesDict.update([string.lower().lstrip() for string in re.split(',|;|/.', note)])
print(notesDict)

Counter({'v případě více zájemců může rk využít pro výběr kupujícího formu aukce.': 609, 'včetně provize': 524, 'včetně právního servisu': 470, 'včetně poplatků': 227, 'včetně dph': 187, 'cena k jednání': 147, '+ provize rk': 73, 'včetně provize rk': 66, 'včetně provize a právního servisu': 58, 'cena včetně provize a právního servisu': 35, 'včetně provize rk a právního servisu': 31, 'na klíč': 22, 'vč. zákl. desky a dph': 22, 'včetně provize a právních služeb': 20, 'včetně právního servisu a provize rk': 19, 'cena včetně provize rk': 18, 'cena včetně 1% provize': 18, 'vč. provize rk': 17, 'včetně provize rk a kompletního právního servisu. . v případě více zájemců může rk využít pro výběr kupujícího formu aukce.': 17, 'bez poplatků': 15, 'cena včetně provize': 14, 'včetně poplatků a právního servisu': 14, 'lze financovat hypotečním úvěrem': 12, 'právního servisu': 12, 'cena včetně provize a právních služeb': 12, 'včetně provize rk a právních služeb': 12, 'vč. provize a právních služeb':

The dictionary above sorts the types of notes found in 'Note on price' variable by frequency of occurance.

In [6]:
df['Note on price'].fillna('Nothing', inplace=True)
df.loc[df['Note on price'].str.contains('poplatků', re.IGNORECASE), ['Note on price']]

,Note on price
2,"včetně DPH, včetně poplatků, včetně provize, včetně právního servisu"
88,"včetně poplatků, včetně provize, včetně právního servisu, cena k jednání"
146,"včetně poplatků, včetně provize, včetně právního servisu, cena k jednání"
161,"16 800 000, -, včetně DPH, včetně poplatků, včetně provize, včetně právního servisu"
162,"2 500 000, - + provize RD + DPH, včetně DPH, včetně poplatků, včetně provize, včetně právního servisu"
...,...
5907,"včetně DPH, včetně poplatků, včetně provize, včetně právního servisu"
5962,"bez poplatků, včetně provize, včetně právního servisu"
5969,"včetně DPH, včetně poplatků, včetně provize, včetně právního servisu, cena k jednání"
5977,"včetně poplatků, včetně provize, včetně právního servisu"


Next I will try to extract from the 'Note on price' variable the information about whether the house price includes comission, legal fees, vat, fees and if the price is negotiable.

In [7]:
def followsPattern(text, pattern):
        return bool(re.search(pattern, text.lower().lstrip()))
        

notelist = ['provize', 'právního', 'dph', 'poplatků']
patternList = []

for note in notelist:
    patternList.append(re.compile('.*(včetně|\+)\s?.*{}.*'.format(note)))
patternList.append(re.compile('.*cena k jednání'))
notelist.append('cena k jednání')

for note,patternToFollow in zip(notelist,patternList):
    df[note] = df.loc[df['Note on price'].apply(followsPattern, pattern=patternToFollow),'Note on price'].map(lambda x: 'True', na_action='ignore')

### Data separation into training set and testing set

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df,df['price'], test_size=0.1, random_state=0)

In [9]:
X_train.shape, X_test.shape

((4910, 75), (546, 75))

## Missing values

### Categorical Values

In [10]:
## First lets handle categorical features which are missing

def percentageMissing(feature):
    missingValues = X_train[feature].apply(lambda x: 1 if pd.isna(x) else 0).sum()
    return round((missingValues/len(X_train.axes[0])) * 100, 3)

def getMissingFeaturesDict(featuresWithNan):
    missingFeaturesDict = {}
    for feature in featuresWithNan:
        percentage = percentageMissing(feature)
        missingFeaturesDict[feature] = percentage
    return sorted(missingFeaturesDict.items(), key=lambda x: x[1], reverse=True)

featuresWithNan=[feature for feature in X_train.columns if X_train[feature].isnull().sum() > 0 and X_train[feature].dtypes=='O']
missingFeaturesDict = getMissingFeaturesDict(featuresWithNan)

for index, feature in enumerate(missingFeaturesDict):
    print('Variable "{}"" has {}% of its values missing'.format(missingFeaturesDict[index][0], missingFeaturesDict[index][1]))
    

Variable "Cena"" has 100.0% of its values missing
Variable "Starting price"" has 100.0% of its values missing
Variable "Expert opinion"" has 100.0% of its values missing
Variable "Minimum bid"" has 100.0% of its values missing
Variable "Auction principal"" has 100.0% of its values missing
Variable "Type of auction"" has 100.0% of its values missing
Variable "Place of auction"" has 100.0% of its values missing
Variable "The date of the auction"" has 100.0% of its values missing
Variable "Auction decree"" has 100.0% of its values missing
Variable "Expert opinion.1"" has 100.0% of its values missing
Variable "Minimum purchase price"" has 100.0% of its values missing
Variable "The date of the 1st inspection"" has 100.0% of its values missing
Variable "The date of the 2nd inspection"" has 100.0% of its values missing
Variable "Inspection date to"" has 99.898% of its values missing
Variable "Inspection date"" has 99.409% of its values missing
Variable "Date of completion of construction"" ha

In [11]:
print(X_train.shape)
cols = X_train.columns[X_train.isnull().mean()>0.98]
X_train.drop(cols, axis=1, inplace = True)
print(X_train.shape)

(4910, 75)
(4910, 54)


This deletes the columns that have more than 98% missing values.

In [12]:
catFeaturesWithNan=[feature for feature in X_train.columns if X_train[feature].isnull().sum() > 0 and X_train[feature].dtypes=='O']
missingFeaturesDict = getMissingFeaturesDict(catFeaturesWithNan)

for index, feature in enumerate(missingFeaturesDict):
    print('Variable "{}"" has {}% of its values missing'.format(missingFeaturesDict[index][0], missingFeaturesDict[index][1]))
    

Variable "Condition"" has 97.902% of its values missing
Variable "Housing costs"" has 97.108% of its values missing
Variable "cena k jednání"" has 96.925% of its values missing
Variable "dph"" has 96.029% of its values missing
Variable "Certificate of energy performance of the building"" has 95.988% of its values missing
Variable "Discounted"" has 95.397% of its values missing
Variable "Original price"" has 95.397% of its values missing
Variable "poplatků"" has 94.705% of its values missing
Variable "Indicator of energy efficiency of the building"" has 93.462% of its values missing
Variable "právního"" has 81.344% of its values missing
Variable "Telecommunication"" has 76.436% of its values missing
Variable "provize"" has 75.519% of its values missing
Variable "Gas"" has 75.193% of its values missing
Variable "Move-in date"" has 70.448% of its values missing
Variable "Store"" has 63.747% of its values missing
Variable "Communication"" has 62.016% of its values missing
Variable "Transpo

In [13]:
def fillNaCatFeature(dataset,catFeaturesWithNan):
    data = X_train.copy()
    data[catFeaturesWithNan]=data[catFeaturesWithNan].fillna('Missing')
    return data

X_train = fillNaCatFeature(X_train,catFeaturesWithNan)

Iputes the cells with missingvalues with a values called 'Missing'

### Numerical variables

In [14]:
df.columns

Index(['link', 'location', 'district', 'HouseType', 'total price', 'Update',
       'Building', 'Object status', 'Location of the house', 'Home type',
       'The floor', 'Built-up area', 'Usable area', 'Floor area', 'Land area',
       'Garden area', 'Store', 'Move-in date', 'Water', 'Heating', 'Gas',
       'Garbage', 'Transport', 'Communication',
       'Energy efficiency of the building', 'Barrier-free', 'Equipment',
       'companyName', 'proximityIndex', 'Index', 'Note on price',
       'Electricity', 'Indicator of energy efficiency of the building',
       'Object location', 'Parking', 'Garage', 'Year of reconstruction',
       'Lift', 'Cena', 'Year of approval', 'Telecommunication',
       'Certificate of energy performance of the building', 'Swimming pool',
       'Number of flats', 'Housing costs', 'Discounted', 'Original price',
       'Pool area', 'Ceiling height', 'Condition', 'Soil construction',
       'Starting price', 'Expert opinion', 'Minimum bid', 'Auction principal

In [15]:
numValsToCat = ['Barrier-free', 'Parking', 'Garage', 'Lift', 'Swimming pool']
for feature in numValsToCat:
    X_train[feature] = X_train[feature].fillna('Missing')

In [17]:
numFeatures = [feature for feature in X_train.columns if X_train[feature].dtypes != 'O' and feature not in ['Index']]
X_train[numFeatures]

,Built-up area,Usable area,Floor area,Land area,Garden area,proximityIndex,Year of reconstruction,Year of approval,price
1805,NaN,190,NaN,1139.0,NaN,2184.533333,NaN,NaN,2260000
2372,NaN,81,NaN,366.0,NaN,1718.454545,NaN,NaN,5995000
3986,344.0,277,NaN,2380.0,2000.0,1531.888889,NaN,NaN,28900000
2482,NaN,299,NaN,2286.0,NaN,1880.714286,NaN,NaN,24950000
754,35.0,35,NaN,169.0,NaN,2068.214286,NaN,NaN,1650000
...,...,...,...,...,...,...,...,...,...
5407,NaN,35,NaN,1431.0,NaN,636.200000,NaN,NaN,1785000
3517,NaN,118,NaN,337.0,NaN,3310.823529,NaN,NaN,2950000
1800,142.0,397,397.0,1170.0,1170.0,753.277778,2006.0,NaN,13490000
2811,95.0,80,80.0,233.0,138.0,539.000000,NaN,NaN,2990000


In [18]:
# Now lets handle numerical features which are missing
numFeaturesWithNan=[feature for feature in numFeatures if X_train[feature].isnull().sum()>1]    
missingFeaturesDict = getMissingFeaturesDict(numFeaturesWithNan)

for index, feature in enumerate(missingFeaturesDict):
    print('Variable "{}"" has {}% of its values missing'.format(missingFeaturesDict[index][0], missingFeaturesDict[index][1]))

Variable "Year of approval"" has 93.727% of its values missing
Variable "Year of reconstruction"" has 90.041% of its values missing
Variable "Garden area"" has 73.238% of its values missing
Variable "Floor area"" has 70.692% of its values missing
Variable "Built-up area"" has 33.666% of its values missing
Variable "Land area"" has 0.061% of its values missing


### Inpute data to numerical variables

In [19]:
X_train[numFeaturesWithNan]

,Built-up area,Floor area,Land area,Garden area,Year of reconstruction,Year of approval
1805,NaN,NaN,1139.0,NaN,NaN,NaN
2372,NaN,NaN,366.0,NaN,NaN,NaN
3986,344.0,NaN,2380.0,2000.0,NaN,NaN
2482,NaN,NaN,2286.0,NaN,NaN,NaN
754,35.0,NaN,169.0,NaN,NaN,NaN
...,...,...,...,...,...,...
5407,NaN,NaN,1431.0,NaN,NaN,NaN
3517,NaN,NaN,337.0,NaN,NaN,NaN
1800,142.0,397.0,1170.0,1170.0,2006.0,NaN
2811,95.0,80.0,233.0,138.0,NaN,NaN


In [20]:
for feature in numFeaturesWithNan:
    # There are many outliers in the dataset, therefore I will use the median.
    median=X_train[feature].median()
    
    # create a new feature to capture nan values
    X_train[feature + 'Nan'] = np.where(X_train[feature].isnull(), 1, 0)
    X_train[feature].fillna(median, inplace=True)

In [21]:
X_train[numFeaturesWithNan]

,Built-up area,Floor area,Land area,Garden area,Year of reconstruction,Year of approval
1805,132.0,142.0,1139.0,489.5,2017.0,2012.0
2372,132.0,142.0,366.0,489.5,2017.0,2012.0
3986,344.0,142.0,2380.0,2000.0,2017.0,2012.0
2482,132.0,142.0,2286.0,489.5,2017.0,2012.0
754,35.0,142.0,169.0,489.5,2017.0,2012.0
...,...,...,...,...,...,...
5407,132.0,142.0,1431.0,489.5,2017.0,2012.0
3517,132.0,142.0,337.0,489.5,2017.0,2012.0
1800,142.0,397.0,1170.0,1170.0,2006.0,2012.0
2811,95.0,80.0,233.0,138.0,2017.0,2012.0


### Normaliztion of numerical data

In [22]:
numFeatures=[feature for feature in X_train.columns if not bool(re.search(r'Nan$', feature)) 
             and X_train[feature].dtypes!='O' and feature not in ['Index']] 
for feature in numFeatures:
    X_train[feature]=np.log(X_train[feature])
X_train[numFeatures].head(5)

,Built-up area,Usable area,Floor area,Land area,Garden area,proximityIndex,Year of reconstruction,Year of approval,price
1805,4.882802,5.247024,4.955827,7.037906,6.193384,7.689158,7.609367,7.606885,14.630875
2372,4.882802,4.394449,4.955827,5.902633,6.193384,7.449181,7.609367,7.606885,15.606436
3986,5.840642,5.624018,4.955827,7.774856,7.600902,7.334257,7.609367,7.606885,17.179352
2482,4.882802,5.700444,4.955827,7.734559,6.193384,7.539407,7.609367,7.606885,17.032384
754,3.555348,3.555348,4.955827,5.129899,6.193384,7.634441,7.609367,7.606885,14.316286


### Feature scaling

In [23]:
for feature in numFeatures:
    print('{}; min: {}; max: {}'.format(feature, X_train[feature].min(), X_train[feature].max()))

Built-up area; min: 0.0; max: 9.18091156128537
Usable area; min: 0.0; max: 10.699213884343516
Floor area; min: 0.0; max: 9.295692039075686
Land area; min: 0.0; max: 13.666759938199
Garden area; min: 0.0; max: 11.33857207782545
proximityIndex; min: -inf; max: 8.36138963118295
Year of reconstruction; min: 0.6931471805599453; max: 9.91457584952155
Year of approval; min: 7.495541943884256; max: 7.6128310304073565
price; min: 11.736069016284437; max: 18.675322962325946


In [24]:
X_train.loc[X_train['proximityIndex'] == X_train['proximityIndex'].min(), ['proximityIndex', 'link']]

,proximityIndex,link
1348,-inf,https://www.sreality.cz/detail/prodej/dum/rodinny/rocov-rocov-/1416616540
1324,-inf,https://www.sreality.cz/detail/prodej/dum/rodinny/zatec-zatec-jana-herbena/2162865740


In [25]:
X_train.loc[X_train['proximityIndex'] == X_train['proximityIndex'].min(), ['proximityIndex']] = X_train['proximityIndex'].median()

Substitute proximity index with '-inf' with median

In [26]:
scaler=MinMaxScaler()
X_train[numFeatures] = scaler.fit_transform(X_train[numFeatures])

X_train[numFeatures]

,Built-up area,Usable area,Floor area,Land area,Garden area,proximityIndex,Year of reconstruction,Year of approval,price
1805,0.531843,0.490412,0.533132,0.514965,0.546223,0.814903,0.750016,0.9493,0.417164
2372,0.531843,0.410726,0.533132,0.431897,0.546223,0.748827,0.750016,0.9493,0.557750
3986,0.636172,0.525648,0.533132,0.568888,0.670358,0.717183,0.750016,0.9493,0.784419
2482,0.531843,0.532791,0.533132,0.565939,0.546223,0.773670,0.750016,0.9493,0.763240
754,0.387254,0.332300,0.533132,0.375356,0.546223,0.799837,0.750016,0.9493,0.371829
...,...,...,...,...,...,...,...,...,...
5407,0.531843,0.332300,0.533132,0.531664,0.546223,0.475224,0.750016,0.9493,0.383162
3517,0.531843,0.445891,0.533132,0.425857,0.546223,0.929391,0.750016,0.9493,0.455560
1800,0.539797,0.559287,0.643732,0.516930,0.623073,0.521736,0.749423,0.9493,0.674624
2811,0.496016,0.409565,0.471404,0.398854,0.434557,0.429572,0.750016,0.9493,0.457501


In [27]:
X_train.to_csv('TrainingData.csv')
X_test.to_csv('TestingData.csv')

In [28]:
numFeatures = [feature for feature in X_train.columns if X_train[feature].dtypes != 'O' and feature not in ['Index']]
X_train[numFeatures]

,Built-up area,Usable area,Floor area,Land area,Garden area,proximityIndex,Year of reconstruction,Year of approval,price,Built-up areaNan,Floor areaNan,Land areaNan,Garden areaNan,Year of reconstructionNan,Year of approvalNan
1805,0.531843,0.490412,0.533132,0.514965,0.546223,0.814903,0.750016,0.9493,0.417164,1,1,0,1,1,1
2372,0.531843,0.410726,0.533132,0.431897,0.546223,0.748827,0.750016,0.9493,0.557750,1,1,0,1,1,1
3986,0.636172,0.525648,0.533132,0.568888,0.670358,0.717183,0.750016,0.9493,0.784419,0,1,0,0,1,1
2482,0.531843,0.532791,0.533132,0.565939,0.546223,0.773670,0.750016,0.9493,0.763240,1,1,0,1,1,1
754,0.387254,0.332300,0.533132,0.375356,0.546223,0.799837,0.750016,0.9493,0.371829,0,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407,0.531843,0.332300,0.533132,0.531664,0.546223,0.475224,0.750016,0.9493,0.383162,1,1,0,1,1,1
3517,0.531843,0.445891,0.533132,0.425857,0.546223,0.929391,0.750016,0.9493,0.455560,1,1,0,1,1,1
1800,0.539797,0.559287,0.643732,0.516930,0.623073,0.521736,0.749423,0.9493,0.674624,0,0,0,0,0,1
2811,0.496016,0.409565,0.471404,0.398854,0.434557,0.429572,0.750016,0.9493,0.457501,0,0,0,0,1,1
